In [6]:
from hhl_openai_agents_sdk.models import get_gemini
from dotenv import load_dotenv
import nest_asyncio
import logfire
import os

load_dotenv()

# make sure we can run async code in jupyter
nest_asyncio.apply()

# instrument logfire
logfire.configure(token=os.getenv("LOGFIRE_TOKEN"))
logfire.instrument_openai()

# initialize the model
model = get_gemini()
# model = get_openai()

Logfire project URL: ]8;id=386498;https://logfire.pydantic.dev/gaohe2z/hhl-openai-agents-sdk\https://logfire.pydantic.dev/gaohe2z/hhl-openai-agents-sdk]8;;\


In [10]:
from agents import Agent, handoff
from agents.extensions import handoff_filters
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX


faq_agent = Agent(name="FAQ agent", model=model)

faq_agent_handoff = handoff(
    agent=faq_agent,
    on_handoff=lambda x: print("FAQ Handoff called"),
    input_filter=handoff_filters.remove_all_tools,
)

sales_agent = Agent(name="Sales agent", model=model, instructions="""
You are a helpful sales agent. You help customers with their product inquiries. Based on the conversation history you
should be able to answer the customer questions without having to ask them the questions customer already provided to you.
""")

sales_agent_handoff = handoff(
    agent=sales_agent,
    on_handoff=lambda x: print("Sales Handoff called"),
)

starter = Agent(
    name="Customer Agent",
    model=model,
    instructions="You are a helpful customer agent. You help customers navigate their request to the right agent. "
    "If customer is asking a question, navigate them to the FAQ agent. "
    "If customer is looking for a product, navigate them to the sales agent.",
    handoff_description=RECOMMENDED_PROMPT_PREFIX,
    handoffs=[faq_agent_handoff, sales_agent_handoff],
)

In [ ]:
from agents import Runner
async def main():
    input_text = "I want to buy a new phone, my budget is $1000."
    result = await Runner.run(starter, input_text)
    print(result.final_output)
    input_text = "How much does the iPhone 15 cost?"
    inputs = result.to_input_list() + [{"role": "user", "content": input_text}]
    result = await Runner.run(starter, inputs)
    print(result.final_output)

In [14]:
await main()

22:19:38.433 Chat Completion with 'gemini-2.0-flash' [LLM]
Sales Handoff called
22:19:38.938 Chat Completion with 'gemini-2.0-flash' [LLM]
Hi there! I'm happy to help you find a new phone. To give you the best recommendations, could you tell me a bit about what you're looking for? For example:

*   **What's your budget?**
*   **What features are most important to you?** (e.g., camera quality, battery life, screen size, specific apps)
*   **Do you have a preferred brand or operating system?** (e.g., Apple, Android)

Once I have a better understanding of your needs, I can suggest some phones that would be a good fit.



UserError: Unhandled item type or structure: how much does iPhone X cost?